In [ ]:
import cdsapi
import calendar

c = cdsapi.Client()

years = ['2018','2019','2020','2021','2022','2023']

for year in years:
    for month in range(1, 13):
        days_in_month = calendar.monthrange(int(year), month)[1]

        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': [
                    '2m_temperature',
                    '2m_dewpoint_temperature',
                ],
                'year': year,
                'month': f"{month:02d}",
                'day': [f"{d:02d}" for d in range(1, days_in_month + 1)],
                'time': [f"{h:02d}:00" for h in range(24)],
                'area': [
                    28.9, 76.8, 28.4, 77.4  
                ],
                'format': 'netcdf',
            },
            f'delhi_weather_{year}_{month:02d}.nc'
        )

        print(f"Downloaded {year}-{month:02d}")


/Users/panavdawar/Documents/resilienceAI /.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2026-01-06 21:47:37,586 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2026-01-06 21:47:37,587 INFO Request ID is 74b13edf-8838-4047-bef9-50ed21a3629f
2026-01-06 21:47:37,819 INFO status has been updated to accepted


KeyboardInterrupt: 

In [1]:
import xarray as xr
import pandas as pd
import glob

files = sorted(glob.glob("delhi_weather_*.nc"))
dfs = []

for f in files:
    ds = xr.open_dataset(f)
    ds = ds[['t2m', 'd2m']]
    ds = ds.load()

    df = ds.to_dataframe().reset_index()
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

data = data.rename(columns={
    't2m': 'temp',
    'd2m': 'dewpoint',
    'latitude': 'lat',
    'longitude': 'lon',
    'valid_time': 'datetime'
})

data['temp'] = data['temp'] - 273.15
data['dewpoint'] = data['dewpoint'] - 273.15

data = data.dropna(subset=['temp','dewpoint','lat','lon'])

print(data.head())
print("Rows:", len(data))


    datetime    lat    lon      temp  dewpoint  number expver
0 2018-01-01  28.90  76.80  6.656921  5.852478       0   0001
1 2018-01-01  28.90  77.05  6.940277  6.219910       0   0001
2 2018-01-01  28.90  77.30  7.161743  6.618622       0   0001
3 2018-01-01  28.65  76.80  6.691376  5.641388       0   0001
4 2018-01-01  28.65  77.05  7.003784  6.090027       0   0001
Rows: 473256


In [2]:

if 'time' in data.columns:
    data = data.rename(columns={'time': 'datetime'})
elif 'datetime' not in data.columns:
    data = data.reset_index().rename(columns={'time': 'datetime'})

data['datetime'] = pd.to_datetime(data['datetime'])


data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour


In [3]:
import numpy as np

data = data.dropna()

mask = data['dewpoint'] > data['temp']
data.loc[mask, 'dewpoint'] = data.loc[mask, 'temp']

data = data[(data['temp'] > -50) & (data['temp'] < 60)]
data = data[(data['dewpoint'] > -50) & (data['dewpoint'] < 60)]

print(data.describe())  


                            datetime            lat            lon  \
count                         473256  473256.000000  473256.000000   
mean   2020-12-31 11:30:00.000000256      28.650000      77.050000   
min              2018-01-01 00:00:00      28.400000      76.800000   
25%              2019-07-02 17:45:00      28.400000      76.800000   
50%              2020-12-31 11:30:00      28.650000      77.050000   
75%              2022-07-02 05:15:00      28.900000      77.300000   
max              2023-12-31 23:00:00      28.900000      77.300000   
std                              NaN       0.204124       0.204124   

                temp       dewpoint    number           year          month  \
count  473256.000000  473256.000000  473256.0  473256.000000  473256.000000   
mean       24.867821      17.239885       0.0    2020.499772       6.523962   
min         2.325256      -4.697113       0.0    2018.000000       1.000000   
25%        18.610123      11.809296       0.0    2019

In [ ]:
import numpy as np

T = data['temp']    
Td = data['dewpoint']
RH = 100 * (np.exp((17.625 * Td)/(243.04+Td)) / np.exp((17.625 * T)/(243.04+T)))
data['RH'] = RH.round(2)


def heat_index(T, RH):

    T_F = T * 9/5 + 32
    HI_F = -42.379 + 2.04901523*T_F + 10.14333127*RH - 0.22475541*T_F*RH \
           - 0.00683783*T_F*T_F - 0.05481717*RH*RH + 0.00122874*T_F*T_F*RH \
           + 0.00085282*T_F*RH*RH - 0.00000199*T_F*T_F*RH*RH
    return (HI_F - 32) * 5/9

data['heat_index'] = heat_index(data['temp'], data['RH']).round(1)

print(data[['temp','dewpoint','RH','heat_index']])


            temp  dewpoint         RH  heat_index
0       6.656921  5.852478  94.610001   34.700001
1       6.940277  6.219910  95.169998   33.299999
2       7.161743  6.618622  96.339996   31.600000
3       6.691376  5.641388  93.010002   35.700001
4       7.003784  6.090027  93.910004   34.000000
...          ...       ...        ...         ...
473251  9.042786  8.337189  95.339996   26.700001
473252  8.795746  8.202332  96.059998   26.799999
473253  9.803772  9.223602  96.180000   24.100000
473254  9.134888  8.600861  96.459999   25.600000
473255  8.562225  8.095734  96.889999   26.799999

[473256 rows x 4 columns]


In [87]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


In [88]:
def sanitize_gdf(gdf):
    gdf = gdf.drop(
        columns=[c for c in ['index_left', 'index_right'] if c in gdf.columns],
        errors='ignore'
    )
    gdf = gdf.reset_index(drop=True)
    gdf.index.name = None
    return gdf


districts = gpd.read_file("delhi_districts.geojson").to_crs(epsg=4326)
districts = sanitize_gdf(districts)


In [89]:
data_points = gpd.GeoDataFrame(
    data.copy(),
    geometry=gpd.points_from_xy(data["lon"], data["lat"]),
    crs="EPSG:4326"
)

data_points = sanitize_gdf(data_points)


In [ ]:
data_points = gpd.sjoin(
    data_points,
    districts,
    how="left",
    predicate="intersects"  
)


In [91]:
monthly = (
    data
    .groupby(['year', 'month'], as_index=True)['temp']
    .mean()
    .reset_index()
)


In [92]:
monthly_pivot = monthly.pivot(
    index='month',
    columns='year',
    values='temp'
).sort_index()


In [93]:
import matplotlib
matplotlib.use("Agg")


In [ ]:

months = [int(m) for m in monthly_pivot.index.tolist()]

clean_data = {
    str(year): [float(x) if pd.notna(x) else None for x in monthly_pivot[year].tolist()]
    for year in monthly_pivot.columns
}


In [95]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

for year, temps in clean_data.items():
    plt.plot(
        months,
        temps,
        marker="o",
        label=year
    )

plt.xlabel("Month")
plt.ylabel("Average Temperature (°C)")
plt.title("Monthly Average Temperature in Delhi")
plt.xticks(range(1, 13))
plt.grid(True)
plt.legend(title="Year")

plt.tight_layout()
plt.savefig("monthly_avg_temp.png", dpi=150, format="png")
plt.show()


/var/folders/5c/45x3dg4j3f5ct_z61fnh86nr0000gn/T/ipykernel_20823/1403402590.py:22: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [96]:
import matplotlib
matplotlib.use("Agg")


In [ ]:

data['risk_label'] = 0  # default low risk
high_risk = (data['temp'] > 35) & (data['RH'] > 70)  
data.loc[high_risk, 'risk_label'] = 1



In [ ]:

data['date'] = pd.to_datetime(data['datetime']).dt.date
daily = data.groupby(['date', 'lat', 'lon']).agg({'temp':'mean','RH':'mean','risk_label':'mean'}).reset_index()

sample_day = daily[daily['date'] == pd.to_datetime('2023-06-15').date()]
print(sample_day)


             date    lat    lon       temp         RH  risk_label
17919  2023-06-15  28.40  76.80  33.462532  49.559170         0.0
17920  2023-06-15  28.40  77.05  33.797329  48.568333         0.0
17921  2023-06-15  28.40  77.30  34.137463  47.600002         0.0
17922  2023-06-15  28.65  76.80  33.342510  50.403336         0.0
17923  2023-06-15  28.65  77.05  33.542679  49.837498         0.0
17924  2023-06-15  28.65  77.30  33.481434  49.824585         0.0
17925  2023-06-15  28.90  76.80  33.022198  51.437916         0.0
17926  2023-06-15  28.90  77.05  32.979698  51.522503         0.0
17927  2023-06-15  28.90  77.30  32.863525  51.681667         0.0


In [ ]:
import folium
grid_risk = (
    daily
    .groupby(['lat', 'lon'])['risk_label']
    .mean()
    .reset_index()
)


m = folium.Map(location=[28.7, 77.2], zoom_start=10)

for _, row in sample_day.iterrows():
    color = 'red' if row['risk_label'] >= 0.5 else 'green'
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Temp: {row['temp']:.1f}°C, RH: {row['RH']:.0f}%, Risk: {row['risk_label']:.0f}"
    ).add_to(m)

m.save("delhi_risk_map.html")


In [101]:
import folium

m = folium.Map(location=[28.65, 77.1], zoom_start=10, tiles="cartodbpositron")
for _, row in grid_risk.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        fill=True,
        fill_color=(
            "green" if row['risk_label'] < 0.3 else
            "orange" if row['risk_label'] < 0.6 else
            "red"
        ),
        fill_opacity=0.75,
        popup=f"Risk: {row['risk_label']:.2f}"
    ).add_to(m)
m.save("delhi_grid_risk_map.html")


In [ ]:
import dash
from dash import html, dcc

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H2("Delhi Outdoor Risk Dashboard"),
    html.Div("Interactive map and controls will go here."),
    html.Iframe(id='risk-map', srcDoc=open('delhi_risk_map.html','r').read(), 
                style={'width': '100%', 'height': '600px'})
])

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
from sklearn.model_selection import train_test_split

features = ['temp', 'dewpoint', 'RH', 'heat_index']  
X = data[features].values
y = data['risk_label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

import joblib
joblib.dump(clf, 'rf_model.pkl')


['rf_model.pkl']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


importances = clf.feature_importances_
print("Feature importances:", dict(zip(features, importances.round(3))))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     94652

    accuracy                           1.00     94652
   macro avg       1.00      1.00      1.00     94652
weighted avg       1.00      1.00      1.00     94652

Confusion matrix:
 [[94652]]
Feature importances: {'temp': np.float64(0.235), 'dewpoint': np.float64(0.312), 'RH': np.float64(0.304), 'heat_index': np.float64(0.149)}


/Users/panavdawar/Documents/resilienceAI /.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:407: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.



In [105]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(16, activation='relu', input_shape=(len(features),)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # binary risk (0/1)
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/Users/panavdawar/Documents/resilienceAI /.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225 (900.00 B)

 Trainable params: 225 (900.00 B)

 Non-trainable params: 0 (0.00 B)

In [106]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 8s 716us/step - accuracy: 0.9857 - loss: 0.2148 - val_accuracy: 1.0000 - val_loss: 5.4440e-07
Epoch 2/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 7s 720us/step - accuracy: 1.0000 - loss: 2.2230e-04 - val_accuracy: 1.0000 - val_loss: 7.3315e-07
Epoch 3/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 7s 701us/step - accuracy: 1.0000 - loss: 1.4649e-04 - val_accuracy: 1.0000 - val_loss: 1.0913e-07
Epoch 4/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 6s 684us/step - accuracy: 1.0000 - loss: 7.1557e-05 - val_accuracy: 1.0000 - val_loss: 2.0947e-06
Epoch 5/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 7s 698us/step - accuracy: 1.0000 - loss: 8.0847e-05 - val_accuracy: 1.0000 - val_loss: 5.1606e-07
Epoch 6/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 7s 688us/step - accuracy: 1.0000 - loss: 5.9058e-05 - val_accuracy: 1.0000 - val_loss: 7.8239e-07
Epoch 7/10
9466/9466 ━━━━━━━━━━━━━━━━━━━━ 6s 673us/step - accuracy: 1.0000 - loss: 1.9531e-04 - val_accuracy: 1.0000 - val_loss: 1.0449e-07
Epoch 8/10
9466/9466 ━━━

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"DL model accuracy: {acc:.3f}, loss: {loss:.3f}")


y_pred_dl = (model.predict(X_test) > 0.5).astype(int)


2958/2958 ━━━━━━━━━━━━━━━━━━━━ 2s 513us/step - accuracy: 1.0000 - loss: 1.3959e-07
DL model accuracy: 1.000, loss: 0.000
2958/2958 ━━━━━━━━━━━━━━━━━━━━ 1s 326us/step
